In [5]:
import numpy as np
import pandas as pd
from astropy.coordinates import angular_separation
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=71, Om0=0.27) #initialize a cosmology class with H0=71 and Om0=0.27

BCGs = pd.read_excel(r'../full_BCG_list.xlsx')
BCGs = BCGs.to_numpy(dtype=str)
BCGsClusterNames = BCGs.T[0]
BCGsClusterNames = np.char.strip(BCGsClusterNames)

BCG_RAs = BCGs.T[5]
BCG_DECs = BCGs.T[6]
BCG_Zs = BCGs.T[7]

#get global props data
globalClusters = np.loadtxt(r"../../tools/data/global_properties.csv",delimiter=',',dtype=str)
globalNames = globalClusters.T[0][3:]
globalNames = np.char.strip(globalNames)
globalRAs = dict(zip(globalNames,globalClusters.T[8][3:]))
globalDECs = dict(zip(globalNames,globalClusters.T[9][3:]))
globalZs = dict(zip(globalNames,globalClusters.T[10][3:]))

#get profile K0
profiles = np.loadtxt(r"../../tools/data/ACCEPT2_K0_and_z.csv",delimiter=',',dtype=str)
profilesNames = profiles.T[1][1:]
profilesNames = np.char.strip(profilesNames)
profileK0s = dict(zip(profilesNames,profiles.T[2][1:]))
profileK0errs = dict(zip(profilesNames,profiles.T[3][1:]))

In [6]:
#initialize empty table
table = np.zeros((len(BCGsClusterNames),9),dtype=object)

#set first column to all accept2.0 names
table[:,0]=BCGsClusterNames

In [7]:
for i, name in enumerate(BCGsClusterNames): #goes through each cluster and compares the coords
    coords = np.radians((float(globalRAs[name]),float(globalDECs[name]),float(BCG_RAs[i]),float(BCG_DECs[i]))) #converts coords to radians
    separation = angular_separation(*coords) #uses astropy to calculate
    table[i,1] = np.degrees(separation)*3600 #converts separation to arcseconds
    table[i,2] = (cosmo.kpc_proper_per_arcmin(float(globalZs[name]))*u.arcmin/(60*u.arcsec)).value
    table[i,3] = table[i,2]*table[i,1]
    table[i,4] = float(BCG_Zs[i])
    table[i,5] = np.abs(float(BCG_Zs[i])-float(globalZs[name]))/float(globalZs[name])
    table[i,6] = np.abs(float(BCG_Zs[i])-float(globalZs[name]))/(1+float(globalZs[name]))
    try: table[i,7] = float(profileK0s[name]); table[i,8] = float(profileK0errs[name]) #sees if there is a K0
    except: table[i,7] = np.nan; table[i,8] = np.nan

In [8]:
df = pd.DataFrame(table,columns=['Cluster Name', 'Angular Separation (arcseconds)', 'kpc/arcsec Scale', 'kpc Separation', 'BCG Z', 'Z Separation', 'Z Separation over +1', "K0 (keV cm2)", "K0err (keV cm2)"])
df = df.sort_values('kpc Separation',ascending=False)
df.to_csv("bcg_coords_compare.csv",index=None)